In [1]:
import json
import shutil

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import tiledb
import pdal
from pybabylonjs import Show as show

In [2]:
#!wget -nc "https://github.com/PDAL/data/blob/master/workshop/autzen.laz?raw=true" -O "autzen.laz"

In [3]:
#try:
#    shutil.rmtree("autzen")
#    pass
#except:
#    pass

In [4]:
pipeline1 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen1",chunk_size=10000000)
)

# count = pipeline1.execute()  

In [5]:
pipeline2 = (
  pdal.Reader("autzen.laz") | 
  pdal.Writer.tiledb(array_name="autzen2",x_tile_size=10, y_tile_size=10, z_tile_size=1,chunk_size=10000000)
)

# count = pipeline2.execute() 

In [6]:
pipeline3 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen3",chunk_size=10000000,data_tile_capacity=100000)
)

# count = pipeline3.execute()  

In [7]:
pipeline4 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen4",chunk_size=100000,data_tile_capacity=100000)
)

# count = pipeline4.execute()  

In [8]:
pipeline5 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen5",chunk_size=100000)
)

# count = pipeline5.execute()  

## Point cloud

In [9]:
%%time
with tiledb.open("autzen1") as arr:
    df = arr.df[636800:637800, 851000:853000, 406.14:615.26]

df.head()

CPU times: user 10.3 s, sys: 2.77 s, total: 13 s
Wall time: 1.72 s


,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime,Red,Green,Blue
0,637475.45,851804.30,415.49,1,1,1,1,0,2,-1.0,122,7331,248289.538944,111,112,98
1,637174.99,852014.95,415.49,1,1,1,1,0,2,10.0,122,7332,248680.116914,114,119,113
2,637371.69,851840.65,415.51,1,1,1,0,0,2,-12.0,124,7330,247564.723623,134,136,113
3,637244.90,851873.25,415.51,1,1,1,0,0,2,0.0,124,7331,248288.150659,138,145,124
4,637169.48,851998.29,415.51,1,1,1,1,0,2,-11.0,124,7330,247565.570015,112,114,108


In [10]:
data = {
    'X': df['X'],
    'Y': df['Y'],
    'Z': df['Z'],
    'Red': df['Red'] / 255.0,
    'Green': df['Green'] / 255.0,
    'Blue': df['Blue'] / 255.0
}

In [11]:
show.from_dict(data=data,
                style = 'pointcloud',
                width = 800,
                height = 600,
                z_scale = .2,
                wheel_precision = 50)

BabylonPC(value={'extents': [636800.02, 637799.99, 851000.03, 852999.99, 415.49, 598.15], 'time': None, 'data'…

## MBRS

### Default sorting

In [12]:
show.from_array(array_uri='autzen1',
                style='mbrs',
                width=800,
                height=600,
                z_scale = 0.5,
                wheel_precision = 50)

BabylonMBRS(value={'extents': [635577.79, 639003.73, 848882.15, 853537.66, 406.14, 615.26], 'data': {'Xmin': 0…

### Optimized sorting - visual tuning - 2.5 x faster loading of data from array

In [13]:
show.from_array(array_uri='autzen5',
                style='mbrs',
                width=800,
                height=600,
                z_scale = 0.2)

BabylonMBRS(value={'extents': [635577.79, 639003.73, 848882.15, 853537.66, 406.14, 615.26], 'data': {'Xmin': 0…

In [14]:
%%time
with tiledb.open("autzen1") as arr:
    df = arr.df[636800:637800, 851000:853000, 406.14:615.26]

CPU times: user 10.9 s, sys: 544 ms, total: 11.5 s
Wall time: 1.34 s


In [15]:
%%time
with tiledb.open("autzen5") as arr:
    df = arr.df[636800:637800, 851000:853000, 406.14:615.26]

CPU times: user 3.47 s, sys: 812 ms, total: 4.28 s
Wall time: 582 ms


## 4D 

In [16]:
# %%time
# with tiledb.open("autzen5") as arr:
#     df = pd.DataFrame(arr[636800:637800, 851000:853000, 406.14:615.26])

# data = {
#     'X': df['X'],
#     'Y': df['Y'],
#     'Z': df['Z'],
#     'Red': df['Red'] / 255.0,
#     'Green': df['Green'] / 255.0,
#     'Blue': df['Blue'] / 255.0,
#     'GpsTime': df['GpsTime']}

In [17]:
# show.from_dict(data=data,
#                       style = 'pointcloud',
#                         width = 800,
#                         height = 600,
#                         z_scale = .2,
#                         time = True)